In [ ]:
model_name = "/bask/homes/f/fksv3157/xngs6460-languages/weixuan/model/llama2-7b-chat"  
import torch
import numpy as np
from transformers import AutoTokenizer,AutoModelForCausalLM,GenerationConfig
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,device_map='auto')

In [ ]:
from numpy import *
punc = ['.</s>','.\n','.',';','!',',','?','\n','</s>','<pad>']
def evalone(ans_res,reference):
    ans = ans_res.strip()
    for p in punc:
        ans = ans.replace(p, "")
    if ans.lower() == reference.strip().lower():
        return 1
    else:
        return 0

In [ ]:
import os
import json
def load_xcopa():
    test_data = []
    with open(f"/bask/homes/f/fksv3157/xngs6460-languages/weixuan/data/xcopa/test.en.jsonl", "r", encoding="utf-8") as f:
        lines = json.load(f)
    for line in lines:
        prompt_question,cor_answer = line["input"],line["output"]
        test_data.append([prompt_question,cor_answer])
    return test_data

In [ ]:

import numpy as np
from numpy import *
import tqdm
def baseline(file,test_data):
    EMs = []
    for idx in range(len(test_data)):
        q,a = test_data[idx][0],test_data[idx][1]   
        target_ids = tokenizer(a, return_tensors='pt')['input_ids'].to('cuda')
        inp = tokenizer(f'{q} {a}', return_tensors='pt').to('cuda')
        outputs = model(**inp)
        logits = outputs.logits
        ans = torch.argmax(logits, dim=-1)[:, -target_ids.size(1):-1].squeeze()
        ans_idss = ans.detach().cpu().numpy().tolist()

        textual_ans = tokenizer.decode(ans_idss, skip_special_tokens=True)
        EM = evalone(textual_ans,test_data[idx][1])
        EMs.append(EM)
    EMs = mean(EMs,axis=0)
    print("baseline",EMs)
    return EMs

In [ ]:
test_data = load_xcopa()
res_base = baseline(file,test_data)